In [4]:
import numpy as np

data_careem = np.load("D:\omer\Spatial_Master_Grid_15x19(60_min_resolution).npy", allow_pickle=True)

print("data_careem.shape: ", data_careem.shape)
data_NYC = np.load('D:\\omer\\ST-SSL\\data\\NYCBike1\\train.npz')
for file in data_NYC.files:
    print(file, data_NYC[file].shape)

data_careem.shape:  (2160, 15, 19, 1)
x (3023, 19, 128, 2)
y (3023, 1, 128, 2)
x_offsets (19, 1)
y_offsets (1, 1)


In [14]:
data_NYC["y_offsets"]

array([[1]], dtype=int64)

In [16]:
reshaped_data_careem.shape

(2160, 285, 1)

The train/val/test data is composed of 4 numpy.ndarray objects:

X: input data. It is a 4D tensor of shape (#samples, #lookback_window, #nodes, #flow_types), where # denotes the number sign.
<br>Y: data to be predicted. It is a 4D tensor of shape (#samples, #predict_horizon, #nodes, #flow_types). Note that X and Y are paired in the sample dimension. For instance, (X_i, Y_i) is the i-the data sample with i indexing the sample dimension.
<br>X_offset: a list indicating offsets of X's lookback window relative to the current time with offset 0.
<br>Y_offset: a list indicating offsets of Y's prediction horizon relative to the current time with offset 0.

In [1]:
import networkx as nx
import numpy as np

# Create a 15x19 grid graph
G = nx.grid_2d_graph(15, 19)

# Create an empty adjacency matrix
adj_matrix = np.zeros((15*19, 15*19))

# Iterate over all nodes in the grid
for node in G.nodes:
    # Get the row and column indices of the current node
    row, col = node
    
    # Iterate over the 8 neighbors of the current node
    for i in range(-1, 2):
        for j in range(-1, 2):
            # Skip the current node itself
            if i == 0 and j == 0:
                continue
            
            # Calculate the row and column indices of the neighbor
            neighbor_row = row + i
            neighbor_col = col + j
            
            # Check if the neighbor is within the grid boundaries
            if 0 <= neighbor_row < 15 and 0 <= neighbor_col < 19:
                # Calculate the indices of the current node and the neighbor in the adjacency matrix
                node_index = row * 19 + col
                neighbor_index = neighbor_row * 19 + neighbor_col
                
                # Set the corresponding entry in the adjacency matrix to 1
                adj_matrix[node_index, neighbor_index] = 1

# Print the adjacency matrix
print(adj_matrix)


[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [9]:
np.savez('D:\\omer\\ST-SSL\\data\\Careem\\adj_matrix.npz', adj_matrix=adj_matrix)


In [7]:
reshaped_data_careem = data_careem.reshape(data_careem.shape[0], -1, 1)
print("reshaped_data_careem.shape: ", reshaped_data_careem.shape)  ## #samples, #nodes, demand


reshaped_data_careem.shape:  (2160, 285, 1)


In [18]:
reshaped_data_careem_2 = np.expand_dims(reshaped_data_careem, axis=1)
print("reshaped_data_careem_2.shape: ", reshaped_data_careem_2.shape)  ## #samples, #lookback_window #nodes, demand

reshaped_data_careem_2.shape:  (2160, 1, 285, 1)
